In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D , Dense , Dropout , Input
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input

In [ ]:
import kagglehub

# Download latest version
source = kagglehub.dataset_download("omkargurav/face-mask-dataset")

print("Path to dataset files:", source)

100%|██████████| 163M/163M [00:08<00:00, 20.6MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/omkargurav/face-mask-dataset/versions/1


In [ ]:
source+="/data"
import os
os.listdir(source)

['with_mask', 'without_mask']

In [ ]:
import os
import shutil
import random

def split_data(source , destination , train_size , val_size):
  classes = os.listdir(source)

  for cls in classes:
    path = os.path.join(source , cls)
    print(f"Class {cls} path: {path}")

    files = os.listdir(path)
    random.shuffle(files)

    train_end = int(len(files)*train_size)
    val_end = int(len(files)*(train_size+val_size))

    train_files = files[:train_end]
    val_files = files[train_end:val_end]
    test_files = files[val_end:]

    for split in ['train','val' , 'test']:
      os.makedirs(os.path.join(destination , split , cls) , exist_ok=True)

    def copy_files(img_list , split):
      for img in img_list:
        src = os.path.join(path , img)
        dst = os.path.join(destination , split , cls , img)
        shutil.copy(src , dst)

    copy_files(train_files , 'train')
    copy_files(val_files , 'val')
    copy_files(test_files , 'test')



In [ ]:
split_data(source,"output",0.7,0.2)

Class with_mask path: /root/.cache/kagglehub/datasets/omkargurav/face-mask-dataset/versions/1/data/with_mask
Class without_mask path: /root/.cache/kagglehub/datasets/omkargurav/face-mask-dataset/versions/1/data/without_mask


In [ ]:


finalpath = "output"
train = os.path.join(finalpath , "train")
val = os.path.join(finalpath , "val")
test = os.path.join(finalpath , "test")

In [ ]:
class FaceMaskDetector:
  def __init__(self , image_size=(224,224) , freeze_layers=True):
    self.image_size = image_size
    self.freeze_layers = freeze_layers
    self.model = None


  def preprocessData(self , base_path):
    train = os.path.join(base_path , "train")
    val = os.path.join(base_path , "val")
    test = os.path.join(base_path , "test")

    train_datagen = ImageDataGenerator(
        preprocessing_function = preprocess_input,
        rotation_range=20,
        zoom_range=0.2,
        horizontal_flip=True,
        width_shift_range=0.2,
        height_shift_range=0.2,
    )

    val_datagen = ImageDataGenerator(
        preprocessing_function = preprocess_input
    )

    test_datagen = ImageDataGenerator(
        preprocessing_function = preprocess_input
    )

    train_ds = train_datagen.flow_from_directory(
        train,
        target_size=self.image_size,
        batch_size=32,
        class_mode='binary',
        shuffle=True
    )

    val_ds = val_datagen.flow_from_directory(
        val,
        target_size=self.image_size,
        batch_size=32,
        class_mode='binary',
        shuffle=True
    )
    test_ds = test_datagen.flow_from_directory(
        test,
        target_size=self.image_size,
        batch_size=32,
        class_mode='binary',
        shuffle=True
    )

    return train_ds , val_ds , test_ds



  def build_model(self):

    base_model = MobileNetV2(
        weights='imagenet',
        include_top=False,
        input_shape=self.image_size + (3,)
    )

    if self.freeze_layers:
      for layers in base_model.layers:
        layers.trainable = False


    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)
    output = Dense(1, activation='sigmoid')(x)

    self.model = Model(inputs=base_model.input, outputs=output)

    print("Model build successfully")

  def compile_model(self , learningRate):
    self.model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learningRate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

    print("Model compiled successfully")

  def train(self , train_ds ,val_ds , epochs):
    print("Training started")

    history = self.model.fit(
        train_ds ,
        validation_data=val_ds ,
        epochs=epochs
    )

    print("Training completed")

    return history

  def save_model(self , path):
    self.model.save(path)
    print("Model saved successfully")


  def fine_tuning(self , train_ds , val_ds , unfreeze_last = 20 , learningRate=1e-5 , epochs = 10  ):
    print(f"Unfreezing last {unfreeze_last} layers")
    for layer in self.model.layers[:-unfreeze_last]:
      if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

    self.compile_model(learningRate)

    self.train(train_ds , val_ds , epochs)







In [ ]:
detector = FaceMaskDetector()

train_ds , val_ds , test_ds = detector.preprocessData(finalpath)

Found 5286 images belonging to 2 classes.
Found 1511 images belonging to 2 classes.
Found 756 images belonging to 2 classes.


In [ ]:
detector.build_model()

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Model build successfully


In [ ]:
history = detector.train(train_ds , val_ds , 10)

Training started


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
159/215 ━━━━━━━━━━━━━━━━━━━━ 1:45 2s/step - accuracy: 0.6254 - loss: 0.6655

/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


215/215 ━━━━━━━━━━━━━━━━━━━━ 526s 2s/step - accuracy: 0.6557 - loss: 0.6306 - val_accuracy: 0.9290 - val_loss: 0.3091
Epoch 2/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 507s 2s/step - accuracy: 0.9043 - loss: 0.2986 - val_accuracy: 0.9651 - val_loss: 0.1802
Epoch 3/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 512s 2s/step - accuracy: 0.9482 - loss: 0.1941 - val_accuracy: 0.9765 - val_loss: 0.1277
Epoch 4/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 555s 2s/step - accuracy: 0.9624 - loss: 0.1413 - val_accuracy: 0.9805 - val_loss: 0.1028
Epoch 5/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 505s 2s/step - accuracy: 0.9654 - loss: 0.1287 - val_accuracy: 0.9838 - val_loss: 0.0846
Epoch 6/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 494s 2s/step - accuracy: 0.9701 - loss: 0.1071 - val_accuracy: 0.9849 - val_loss: 0.0751
Epoch 7/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 521s 2s/step - accuracy: 0.9722 - loss: 0.0943 - val_accuracy: 0.9871 - val_loss: 0.0667
Epoch 8/10
215/215 ━━━━━━━━━━━━━━━━━━━━ 509s 2s/step - accuracy: 0.9765 - loss: 0.0844 - val_accuracy: 0.987

In [ ]:
detector.save_model("best_mask_detector.h5")

Model saved successfully
